# Evidencia 1. Flujo de trabajo reproducible del proyecto de ciencia de datos
Pandalytics - Equipo 1

* **A00832444** | Andrea Garza  
* **A01197991** | Hiram Maximiliano Muñoz Ramírez  
* **A00517124** | Erick Orlando Hernández Vallejo  
* **A01197655** | Raúl Isaí Murillo Alemán   
* **A01235692** | David Gerardo Martíne Hidrogo 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model

%matplotlib inline

In [2]:
df = pd.read_excel('ternium-data/Exp_2_Pintura_Estructura Analisis de consumo y .xlsx', sheet_name='Base de datos por pintura')
df

In [3]:
def IQR(df: pd.DataFrame, col_name: str):
    column = df[col_name]
    Q1 = np.percentile(column, 25)
    Q3 = np.percentile(column, 75)
    IQR = Q3 - Q1
    print('Rango de valores:', Q1 - 1.5*IQR, 'a', Q3 + 1.5*IQR, '\n')
    print('Valores atípicos: ')
    sns.boxplot(data=column)
    return df[(column < Q1 - 1.5 * IQR) | (column > Q3 + 1.5 * IQR)][col_name]

In [4]:
IQR(df, 'Real')

In [5]:
IQR(df, 'Teo')

In [6]:
IQR(df, 'Dif')

In [7]:
IQR(df, 'Magnitud')

In [8]:
IQR(df, 'Rendimeinto Std')

In [9]:
IQR(df, 'Metros cuadrados reales')

In [10]:
df_rendimiento_real = IQR(df, 'Rendimeinto Real')
df_rendimiento_real

In [11]:
IQR(df, 'Diferencia de Rendimiento')

### Imputación utilizando LOCF para datos tipo object

Para este ejemplo, convertí en nulo el campo de mes en las filas donde hubiera valores atípicos dentro del rendimiento real. Después, para la imputación de la columna utilice LOCF (Last Observation Carried Forward), es decir reemplazar los nulos por el valor no nulo más reciente antes de su fila. En mi opinión, esta técnica funciona bastante bien para esta base de datos ya que para las columnas de Línea y Mes el valor de los datos no cambia constantemente en cada fila.

In [12]:
print('Meses antes de ser convertidos en nulo: \n',df['Mes'][df['Mes'].index.isin(df_rendimiento_real.index)])

df['Mes'] = df['Mes'].where(~df['Mes'].index.isin(df_rendimiento_real.index), np.nan)
print('\nNulos:\n', df['Mes'][df['Mes'].index.isin(df_rendimiento_real.index)])

df['Mes'].ffill(inplace=True)
print('\nResultado después de aplicar LOCF:\n', df['Mes'][df['Mes'].index.isin(df_rendimiento_real.index)])

### Imputación utilizando regresión para datos numéricos

Para la imputación de datos numéricos inicialmente reemplacé los datos atípicos dentro de la columna de Rendimiento Real con nulos. Planeaba reemplazar todos los nulos con el promedio del Rendimiento Real, pero según lo investigado, esta técnica puede dificultar la estimación de la varianza y el error, por lo que opte por utilizar la regresión. En mi opinión, debido a la correlación de las variables con el Rendimiento Real, en especial Rendimiento std y Diferencia de Rendimiento, la regresión fue una buena elección.

In [13]:
print('Datos iniciales antes de convertir en nulo:\n',df['Rendimeinto Real'][df['Rendimeinto Real'].index.isin(df_rendimiento_real.index)])
df['Rendimeinto Real'] = df['Rendimeinto Real'].where(~df['Rendimeinto Real'].index.isin(df_rendimiento_real.index), np.nan)

In [14]:
X_test = df[df['Rendimeinto Real'].isnull()]
X_test = X_test[['Real','Teo',	'Dif', 'Magnitud', 'Rendimeinto Std', 'Metros cuadrados reales', 'Diferencia de Rendimiento']]

In [15]:
new_df = df.dropna(subset=['Rendimeinto Real'])
new_df = new_df[['Real','Teo',	'Dif', 'Magnitud', 'Rendimeinto Std', 'Metros cuadrados reales', 'Diferencia de Rendimiento', 'Rendimeinto Real']]

In [16]:
y = new_df['Rendimeinto Real']
X = new_df.drop('Rendimeinto Real', axis=1)

In [17]:
lr = linear_model.LinearRegression()
lr.fit(X, y)
y_pred = lr.predict(X_test)
print('Valores calculados para los datos faltantes:\n', pd.Series(y_pred))

In [18]:
df.loc[df['Rendimeinto Real'].isnull(), 'Rendimeinto Real'] = y_pred
print("Valores 'nuevos' en el dataframe: \n", df['Rendimeinto Real'][df['Rendimeinto Real'].index.isin(df_rendimiento_real.index)])
print("\nNulos en la columna de Rendimiento Real: ", df['Rendimeinto Real'].isnull().sum())

### Referencias:

Rutecki, M. (2022, 6 diciembre). *Outlier detection methods!.* Kaggle. https://www.kaggle.com/code/marcinrutecki/outlier-detection-methods
Subrahmanya, S. (2018, 7 junio). *Missing Data Imputation using Regression.* Kaggle. https://www.kaggle.com/code/shashankasubrahmanya/missing-data-imputation-using-regression

Hyun, K. (2013). *The prevention and handling of the missing data.* Korean Journal of Anesthesiology, 64(5), 402–406. https://doi.org/10.4097/kjae.2013.64.5.402

In [19]:
import pandas as pd
import numpy as np

paint_df = pd.read_feather('data/analisis_consumo_pintura.feather')
paint_square_meters_df = pd.read_feather('data/pintura_metros_cuadrados_reales.feather')
paint_performance_df = pd.read_feather('data/rendimiento_pintura.feather')
production_df = pd.read_feather('data/production.feather')

coating_df = pd.read_feather('data/pinturas_revestidos_jul20_ago23.feather')
defects_df=pd.read_feather('data/defects.feather')

## 1 _Pipeline_ de datos
Se definió una _pipeline_ de procesamiento de datos en un _script_ de Python independiente, con el cuál se tiene un proceso replicable para llegar a una base de datos limpia. Esta _pipeline_ consta de 6 diferentes pasos:

- Importación de datos
- Concatenación de resúmenes de producción
- Separación de analisis de líneas de producción y sus defectos
- Selección y renombramiento de columnas
- Eliminación de columnas adicionales
- Conversión a formato _Feather_.

A continuación se muestra un resumen de todo el proceso de la _pipeline_ de datos:

### 1.1 Importación de datos
La importación de datos consiste en importar todos los datos relevantes de los archivos de excel proveídos por Ternium. Hay un total de 12 archivos diferentes.Cada uno de estos archivos contiene varias _worksheets_ dentro. Para la mayoría, solamente necesitamos importar una de estas. Solamente el archivo de Análisis de consumo de pintura requirió importar mas _worksheets_:

- Pinturas y revestidos
    - Pintado 1 UNI Agosto
- Análisis de consumo de pintura
    - Base de datos por pintura
    - Metros
    - Rendimientos INDU
- 10 archivos de resúmenes de producción
    - Resúmen producción
 
### 1.2 Concatenación de resúmenes de producción
Para unificar toda la información de resúmenes de producción, concatenamos todos estas tablas, para así acabar con una base de datos más cohesiva y extensiva con la cual trabajar. Se ignoro el índice original de las 10 tablas, ya que realmente no tenían ningún significado.

### 1.3 Separación de analisis de líneas de producción y sus defectos
Los resúmenes de producción contienen un grupo de columnas que se repiten 5 veces. Estás columnas representan hasta 5 posibles defectos en cada línea de producción. Por lo tanto, estas columnas representan una relación uno a muchos entre una línea de producción y sus posibles defectos. Se definió una función que separa estos grupos de 5 columnas hacia una nueva tabla de defectos, con cada defecto recibiendo una referencia hacia su respectiva línea de producción. Finalmente, se eliminaron las columnas de defectos de la tabla original, ya que ahora se pueden referenciar por medio de una operación de merge con la nueva tabla de defectos.

### 1.4 Tren de pensamiento
Se seleccionaron columnas relevantes de cada base de datos para dejar en la base de datos. Estas columnas fueron renombradas para mejor documentar su propósito. A continuación se tiene una lista de las columnas renombradas para cada base de datos, así como las razones por las cuales las seleccionamos.

#### 1.4.1 Pinturas y revestidos
- **'Denominación objeto' por 'production_line':** Posible identificador para cada línea de producción
- **'Material' por 'paint_id'**: Pintura usada, posible identificador para pinturas
- **'Texto breve de material' por 'paint_name'**: Nombre de la pintura usada, posible identificador para pinturas
- **'Valor var.' por 'monetary_value_usd'**: Valor monetario en dólares, útil para calcular costos
- **'Ctd.total reg.' por 'total_liters_used'**: Litros usados, dato clave
- **'Planta' por 'production_plant'**: Identificación de planta, para filtrar datos
- **'Proveedor' por 'supplier'**: Proveedor, para análisis de datos agrupados
- **'Registrado' por 'date'**: Fecha de los datos, dato clave
- **'Hora' por 'hour'**: Hora de los datos, dato clave
- **'Precio' por 'price_per_liter'**: Precio de cada litro, útil para calcular costos

#### 1.4.2 Análisis de consumo de pintura
- **'Linea' por 'production_line'**: Línea de producción
- **'Mes' por 'month'**: Mes del consumo de pintura, dato clave
- **'Mes num' por 'month_number'**: Número del mes
- **'Pintura' por 'paint'**: Pintura, posible identificador para pinturas
- **'Real' por 'real_consumption'**: Consumo real, dato clave
- **'Teo' por 'theoretical_consumption'**: Consumo tórico, dato clave
- **'Dif' por 'consumption_difference'**: Diferencia de consumos, dato clave
- **'Rendimeinto Std' por 'average_yield'**: Rendimiento promedio, dato clave
- **'Rendimeinto Real' por 'real_yield'**: Rendimiento real, dato clave
- **'Diferencia de Rendimiento' por 'yield_difference'**: Diferencia de rendimientos, dato clave
- **'Metros cuadrados reales' por 'real_produced_square_meters'**: Metros cuadrados reales producidos, dato clave

#### 1.4.3 Rendimientos de pinturas por metro cuadrado
- **'Linea' por 'production_line'**: Identificador para cada línea de producción
- **'Mes' por 'month'**: Mes del dato
- **'Num mes' por 'month_number'**: Numero del mes
- **'Pintura' por 'paint_name'**: Nombre de la pintura
- **'Metros cuadrados reales (m2)' por 'real_square_meters'**: Metros cuadrados de rendimiento de la pintura

#### 1.4.4 Rendimientos de pinturas metros cuadrados por litro
- **'Pintura' por 'paint_name'**: Nombre de la pintura
- **'Clave' por 'paint_code'**: Identificador de la pintura
- **'Rendimiento Canning [m2/L]' por  'paint_performance_m2/l'**: Rendimiento de metros cuadrados por litro

#### 1.4.5 Resumen de producción
- **'Linea' por 'production_line'**: Posible identificador para cada línea de producción
- **'Material Entrada' por 'input_material_code'**: Id del material de entrada, dato clave
- **'Material Salida' por 'output_material_code'**: Id del material de salida, dato clave
- **'Fecha Inicio' por 'start_date'**: Fecha de inicio de la producción, dato clave
- **'Fecha Fin' por 'end_date'**: Fecha de fin de la producción, dato clave
- **'Cliente' por 'client_name'**: Cliente, posible agrupación
- **'Código Clear Inf' por 'inferior_clear_code'**: Código del _clear_ inferior
- **'Código Clear Sup' por 'superior_clear_code'**: Código del _clear_ superior
- **'Ancho 1' por 'width1_mm'**: Ancho 1, dato clave
- **'Ancho 2' por 'width2_mm'**: Ancho 2, dato clave
- **'Ancho 3' por 'width3_8mm'**: Ancho 3, dato clave
- **'Ancho' por 'width_mm'**: Ancho promedio, dato clave
- **'Espesor 1' por 'thickness1_mm'**: Espesor 1, dato clave
- **'Espesor 2' por 'thickness2_mm'**: Espesor 2, dato clave
- **'Espesor 3' por 'thickness3_mm'**: Espesor 3, dato clave
- **'Espesor' por 'thickness_mm'**: Espesor promedio, dato clave
- **'Peso Entrada' por 'input_weight_kg'**: Peso de entrada, dato clave
- **'Peso' por 'weight_kg'**: Peso de salida, dato clave
- **'Largo' por 'length_m'**: largo total, dato clave
- **'Color Inferior' por 'inferior_color_code'**: Código del color inferior
- **'Color Superior' por 'superior_color_code'**: Código del color superior
- **'Primer Superior' por 'superior_primer_code'**: Código del _primer_ superior
- **'Primer Inferior' por 'inferior_primer_code'**: Código del _primer_ inferior
- **'Ruta Teórica' por 'route'**: Ruta teórica de la producción, nos sirve para filtrar las líneas de producción de UNI

#### 1.4.6 Defectos de producción
- **'Codigo defecto' por 'defect_code'**: Posible identificador para tipo de defecto
- **'Defecto' por 'defect_name'**: Nombre de defecto
- **'Ubicacion' por 'location'**: Ubicación en la línea donde sucedió el defecto
- **'Es Contencion' por 'is_containment'**: Si el defecto fue contención de un suceso
- **'Es Prevencion' por 'is_preventive'**: Si el defecto fue prevención de un riesgo
- **'Intensidad' por 'intensity'**: Intensidad del suceso
- **'Cara' por 'face'**: Cara donde sucedió el defecto
- **'Lado' por 'side'**: Lado donde sucedió el defecto
- **'Frecuencia' por 'frequency'**: Frecuencia del defecto
- **'Fecha Registro' por 'register_date'**: Fecha del suceso

### 1.5 Eliminación de columnas adicionales
Una vez seleccionadas y renombradas las columnas relevantes, se excluyeron de las tablas todas las columnas adicionales.

### 1.6 Conversión a formato _feather_
_Feather_ es un format de archivos que permite almacenar tablas o _dataframes_ de una manera eficiente y agnostica al lenguaje. Para el reto, es mucho mas eficiente trabajar con los datos en formato _Feather_ que directamente con los archivos de Excel, por lo cual se decidió utilizar este formato para almacenar los artefactos generados por la _pipeline_ de datos.

El proceso de escribir una tabla a formato _Feather_ automáticamente infiere los tipos de todas las columnas, convirtiendo las columnas númericas a sus respectivos tipos, p. ej. '1.02' a tipo _float_ y '232' a tipo _int_. Cualquier otra columna será convertida a tipo _object_, con las cuales se debe llevar a cabo una conversión manual. Esta conversión es realizada en secciones posteriores de este documento.

## 2 Filtrado de datos
Para cuestiones del reto, solamente es de interés trabajar con datos que sean de la planta Ternium Universidad. Por lo tanto, se necesita filtrar las tablas a que únicamente incluyan valores provenientes de ahi.

### 2.1 Pinturas y revestidos
Para la tabla de pinturas y revestidos, la columna `production_plant` indica a cual planta pertenece cada línea de producción. Por lo tanto, se filtraron valores diferentes a 'Uni'.

In [20]:
coating_df = coating_df[coating_df['production_plant'] == 'Uni']
coating_df                                                                                            

### 2.2 Resumen de producción
Para la tabla de resumen de producción, la columna `route` contiene la ruta teoríca que cada pintura sigue. Para verificar si estos son datos relevantes al caso, fue decidido que se filtraria esta columna con base en las partes de cada ruta. Cada ruta esta estructurada de manera que dice 'planta-planta-planta-planta', separados por guiones. Por lo tanto, se separaron todos los valores y se busco que tuvieran el valor 'UNI' dentro.

In [21]:
routes = production_df['route'].apply(lambda route: all(map(lambda value: 'UNI' in value, route.split('-'))))
production_df = production_df[routes]
production_df

## 3 Verificación de inconsistencias en los datos

### 3.1 Análisis de consumo de pintura
Primero, para todas las columnas de tipo objeto de la base de datos de análisis de consumo de pintura se convirtieron a sus respectivos tipos.
En la tabla principal, hay dos:
- production_line: solo tiene dos valores, por lo que es **categoríca**
- month: es el mes textual, por lo que es **categoríca**
- paint: es el identificador de la pintura, por lo que es un **string**

En la tabla de rendimiento por metro cuadrado, hay 3:
- production_line: solo tiene dos valores, por lo que es **categoríca**
- month: es el mes textual, por lo que es **categoríca**
- paint_name: es el identificador de la pintura, por lo que es un **string**

En la tabla de rendimiento de metro cuadrado por litro, hay 2:
- paint_name: el nombre de la pintura, por lo que es un **string**
- paint_code: el identificador de la pintura, por lo que es un **string**

In [22]:
paint_df['production_line'] = paint_df['production_line'].astype('category')
paint_df['month'] = paint_df['month'].astype('category')
paint_df['paint'] = paint_df['paint'].astype('string')
paint_df.info()

In [23]:
paint_square_meters_df['production_line'] = paint_square_meters_df['production_line'].astype('category')
paint_square_meters_df['month'] = paint_square_meters_df['month'].astype('category')
paint_square_meters_df['paint_name'] = paint_square_meters_df['paint_name'].astype('string')
paint_square_meters_df.info()

In [24]:
paint_performance_df['paint_name'] = paint_performance_df['paint_name'].astype('string')
paint_performance_df['paint_code'] = paint_performance_df['paint_code'].astype('string')
paint_performance_df.info()

#### 3.1.1 Diferencia entre consumo real y teórico

Verificamos que la columna de diferencia de consumo se calculara correctamente mediante una serie que contiene la resta del consumo real menos el consumo teórico y comparamos esta diferencia con la de la columna del archivo para verificar si todos los registro son correctos.

In [25]:
calculated_consumption_difference = paint_df['real_consumption'] - paint_df['theoretical_consumption']
if paint_df['consumption_difference'].equals(calculated_consumption_difference):
    print('La columna "consumption_difference" se calculó correctamente.')
else:
    print('La columna "consumption_difference" no se calculó correctamente.')

#### 3.1.2 Metros cuadrados reales

Verificamos que los metros cuadrados reales se calcularan correctamente en el dataframe de análisis de consumo de pintura por medio de un dataframe generado por mmedio de un merge con el dataframe de los metros cuadrados de cada pintura para localizar aquellos puntos en los que tanto la línea de producción, el mes y el nombre de la pintura coincidan para comparar los metros cuadrados de cada dataframe

In [26]:
paint_merged_df = pd.merge(paint_df, paint_square_meters_df, how='left', left_on=['production_line', 'month', 'paint'], right_on=['production_line', 'month', 'paint_name'])
if paint_merged_df['real_produced_square_meters'].equals(paint_merged_df['real_square_meters']):
    print('Los metros cuadrados se calcularon correctamente.')
else:
    print('Los metros cuadrados no se calcularon correctamente.')

#### 3.1.3 Rendimiento Estandar VS Rendimiento Real

Se calculó el nuevo rendimiento std utilizando el promedio del rendimientos por pintura, en vez de utilizar el primer valor encontrado en la hoja de rendimientos. Después, se agrego una columna new_average_yield (nuevo rendimiento std) dentro pintura_df. 

In [27]:
new_performance = paint_performance_df[['paint_name','paint_performance_m2/l']].groupby('paint_name', as_index=False).mean()
paint_df = paint_df.merge(new_performance, left_on='paint', right_on='paint_name')
paint_df.rename(columns = { 'paint_performance_m2/l': 'new_average_yield'}, inplace=True)
paint_df.drop(['paint_name'], axis=1, inplace=True)
paint_df

Se calculo el rendimiento real usando la fórmula que ternium nos proporcionó (metros cuadrados reales / rendimiento real), en donde se almacena en una variable. Después, generamos la diferencia entre lo que nosotros calculamos con los datos dentro de la base de datos. Finalmente, el código filtra los datos y verifica si hay diferencias mayores a 0 y de ser así las imprime, si no imprime que no hay.

In [28]:
#Rendimiento real
calculated_real_yield =  paint_df['real_produced_square_meters'] / paint_df[ 'real_consumption']
real_yield_difference =  paint_df['real_yield'] - calculated_real_yield

#Checa si hay valores mayor a 0
ry_differences = real_yield_difference[real_yield_difference > 0]
if not ry_differences.empty:
    print(differences)
else:
    print("Real Yield : No differences found.")

Aquí, se generó una nueva variable en donde almacena la diferencia entre rendimiento real (proporcionado en la bs) y el rendimiento std (utilizando el promedio). Igual que el código anterior si encontró dato mayor a 0 los imprime, de no ser así imprime un mensaje comentando que no hay diferencias.

En este resultado podemos ver como si encontró diferencias entre real y el nuuevo rendimiento std, donde varían de decimales hasta enteros.

In [29]:
#Rendimiento std
##rendimiento real menos rendimineto estandar actualizado
average_yield_difference = paint_df['real_yield'] - paint_df['new_average_yield'] 

#Checa si hay valores mayor a 0
ay_differences = average_yield_difference[average_yield_difference > 0]
if not ay_differences.empty:
    print(ay_differences)
else:
    print("Average Yield : No differences found.")

### 3.2 Resumen de producción
Primero se convirtieron los datos de tipo object del resumen de producción. Se encontrarón 10 columnas:
- input_material_code: codigo del material de entrada, **string**
- output_material_code: material de salida, **string**
- client_name: nombre de cliente, **string**
- inferior_clear_code: nombre de _clear_ inferior, **string**
- superior_clear_code:nombre de _clear_ superior, **string**
- inferior_color_code: nombre de color inferior, **string**
- superior_color_code: nombre de color superior, **string**
- superior_primer_code: nombre de _primer_ superior, **string**
- inferior_primer_code: nombre de _primer_ inferior, **string**
- route: ruta a tomar, **string**

In [30]:
production_df['input_material_code'] = production_df['input_material_code'].astype('string')
production_df['output_material_code'] = production_df['output_material_code'].astype('string')
production_df['client_name'] = production_df['client_name'].astype('string')
production_df['inferior_clear_code'] = production_df['inferior_clear_code'].astype('string')
production_df['superior_clear_code'] = production_df['superior_clear_code'].astype('string')
production_df['inferior_color_code'] = production_df['inferior_color_code'].astype('string')
production_df['superior_color_code'] = production_df['superior_color_code'].astype('string')
production_df['superior_primer_code'] = production_df['superior_primer_code'].astype('string')
production_df['inferior_primer_code'] = production_df['inferior_primer_code'].astype('string')
production_df['route'] = production_df['route'].astype('string')
production_df.info()

#### 3.2.1 Promedio de anchos

En el resultado podemos observar como si llego a detectar diferencias dentro del promedio del espesor, la cantidad de diferencias salió alta debido al margen de diferencia de los decimales. Al checar con un valor mayor a 0 sale un margen alto de datos.

In [31]:
# Calculate average width
average_width_mm = production_df[['width1_mm', 'width2_mm', 'width3_mm']].mean(axis=1)
#print(average_width_mm)
# Calculate real width difference
real_width_mm = production_df['width_mm'] - average_width_mm

# Check for differences
aw_differences = real_width_mm[real_width_mm >  0 ]                                                                                              
if not aw_differences.empty:
    print(aw_differences)
else:
    print("No Differences")

Si nos vamos a diferencia mayor a 1, disminuye el margen de diferencias. 

In [32]:
# Check for differences
aw_differences = real_width_mm[real_width_mm >  1 ]
if not aw_differences.empty:
     print('Differences found: \n' , aw_differences)
else:
    print("No Differences")

#### 3.2.2 Promedio de espesores

En el resultado podemos observar como si llego a detectar diferencias dentro del promedio del espesor, la cantidad de diferencias salió alta debido al margen de diferencia de los decimales. Al checar con un valor mayor a 0 sale un margen alto de datos.

In [33]:
average_thickness_mm =production_df[['thickness1_mm', 'thickness2_mm', 'thickness3_mm']].mean(axis=1)

difference_thickness_mm =  production_df['thickness1_mm'] - average_thickness_mm 

# Check for differences
t_differences = difference_thickness_mm[difference_thickness_mm >  0]
if not t_differences.empty:
     print('Differences found: \n' , t_differences)
else:
    print("No Differences")

Si nos vamos a diferencia mayor a 1, disminuye el margen de diferencias. 

In [34]:
# Check for differences
t_differences = difference_thickness_mm[difference_thickness_mm >  1]
if not t_differences.empty:
    print('Differences found: \n' , t_differences)
else:
    print("No Differences")

#### 3.2.3 Colores repetidos

Verificamos si es que tenemos renglones/registros en dónde aparezcan los mismos colores. Esto es en las columnas "Inferior Clear Code"," Superior Clear Code",  "Clear Inferior",  "Clear superior", "Color Inferior",  "Color Superior", "Primer Superior" y "Primer Inferior".

Esa verificación fue llevada a cabo al filtrar el dataframe para solo quedarnos con renglones/registros en donde existan pinturas repetidas, se ignoraron los valores None en cada renglon/Registro.

Al tener el dataframe filtrado procedemos a obtener información relevante de los procesos de pintado como sacar las tipos de pinturas más utilizadas en cada proceso del pintado; estás siendo clear, superior y color


In [35]:
#Columnas para evaluar pinturas repetidas
relevant_columns = ['inferior_clear_code', 'superior_clear_code', 'inferior_color_code', 'superior_color_code', 'superior_primer_code', 'inferior_primer_code']

#Creamos copia de producción
filtered_df = production_df.copy()

# Función para verificar si hay valores repetidos en un registro específico                                                                                                                                 
def check_duplicate_colors(row):
    values = [] # se crea una lista de valores en las columnas
    for col_name in relevant_columns: #revisamos las columnas relevantes                                                                                                                                                                                                                           
        if(row[col_name] != None): # si la columna tiene un identificador de pintura
            values.append(row[col_name]) #lo agregamos a la lista

    # si todos los valores son unicos y la lista no esta vacía, significa que se llevaron a cabo procesos de pintado y cada uno utilizo pinturas distintas
    return len(values) != len(set(values)) and len(values) > 0

# Utilizamos la función con el dataframe filtrado
duplicate_indices = filtered_df[filtered_df.apply(check_duplicate_colors, axis=1)].index.tolist()

# Imprimir indices de renglones con valroes repetidos
#print("Indices of rows with repeated values in 'inferior_color_code', 'superior_color_code', 'superior_primer_code', and 'inferior_primer_code':", duplicate_indices)

# Renglones con valores repetidos en las columnas de interes
rows_with_repeated_values = filtered_df.loc[duplicate_indices, ['inferior_clear_code', 'superior_clear_code', 'inferior_color_code', 'superior_color_code', 'superior_primer_code', 'inferior_primer_code']]

rows_with_repeated_values

Información de pinturas existentes en los procesos

In [36]:
# Inicializamos diccionarios para almacenar las pinturas únicas por proceso
unique_paints_by_process = {
    'clear': set(),
    'color': set(),
    'primer': set()
}

# Iteramos sobre cada columna para identificar las pinturas únicas y su proceso correspondiente
for col in rows_with_repeated_values.columns:
    if 'clear' in col:
        unique_paints_by_process['clear'].update(rows_with_repeated_values[col].dropna().unique())
    elif 'color' in col:
        unique_paints_by_process['color'].update(rows_with_repeated_values[col].dropna().unique())
    elif 'primer' in col:
        unique_paints_by_process['primer'].update(rows_with_repeated_values[col].dropna().unique())

# Convertir sets a listas para una manipulación más fácil si es necesario
for process in unique_paints_by_process:
    unique_paints_by_process[process] = list(unique_paints_by_process[process])

# Mostrar las pinturas únicas por proceso
for process, paints in unique_paints_by_process.items():
    print(f"Proceso de {process}:")
    for paint in paints:
        print(f" - {paint}")

# Para obtener una lista general de todas las pinturas únicas, sin importar el proceso
all_unique_paints = set().union(*unique_paints_by_process.values())
print("\nLista general de todas las pinturas únicas:")
for paint in all_unique_paints:
    print(f" - {paint}")

# Determinar en qué proceso(s) se utiliza cada pintura única
paints_process_usage = {paint: [] for paint in all_unique_paints}
for paint in all_unique_paints:
    for process, paints in unique_paints_by_process.items():
        if paint in paints:
            paints_process_usage[paint].append(process)

# Mostrar en qué proceso(s) se utiliza cada pintura
print("\nUso de pinturas en procesos:")
for paint, processes in paints_process_usage.items():
    print(f" - Pintura {paint} se utiliza en el(los) proceso(s) de {' y '.join(processes)}.")


Información sobre colores repetidos en cada tipo de proceso

In [37]:
# Definimos diccionarios para agrupar los identificadores de pinturas por tipo de proceso
clear_codes = {'inferior_clear_code', 'superior_clear_code'}
primer_codes = {'inferior_primer_code', 'superior_primer_code'}
color_codes = {'inferior_color_code', 'superior_color_code'}

# Creamos DataFrames vacíos para cada tipo de proceso
clear_paints = pd.DataFrame(columns=['Paint Code', 'Process Type', 'Count'])
primer_paints = pd.DataFrame(columns=['Paint Code', 'Process Type', 'Count'])
color_paints = pd.DataFrame(columns=['Paint Code', 'Process Type', 'Count'])

# Función para contar y agregar pinturas por tipo de proceso
def add_paint_count(row, process_dict, process_name, df):
    for col_name in process_dict:
        paint_code = row[col_name]
        if pd.notna(paint_code):  # Verificamos que el código de pintura no sea NaN
            if not df[df['Paint Code'] == paint_code].empty:
                df.loc[df['Paint Code'] == paint_code, 'Count'] += 1
            else:
                df = pd.concat([df, pd.DataFrame([[paint_code, process_name, 1]], columns=df.columns)], ignore_index=True)
    return df

# Recorremos el DataFrame para contar y agrupar pinturas por proceso
for index, row in rows_with_repeated_values.iterrows():
    clear_paints = add_paint_count(row, clear_codes, 'Clear', clear_paints)
    primer_paints = add_paint_count(row, primer_codes, 'Primer', primer_paints)
    color_paints = add_paint_count(row, color_codes, 'Color', color_paints)

# Unimos los resultados para tener un resumen completo
all_paints = pd.concat([clear_paints, primer_paints, color_paints]).reset_index(drop=True)

# Mostramos las pinturas existentes y su uso por proceso
print("Summary of Paint Use by Process:")
#print(all_paints.sort_values(by=['Process Type', 'Count'], ascending=[True, False]))

# Para ver cuáles pinturas son las más usadas en cada proceso
print("\nMost Used Paints by Process:")
for process_type, group_df in all_paints.groupby('Process Type'):
    most_used = group_df.loc[group_df['Count'].idxmax()]
    print(f"{process_type}: {most_used['Paint Code']} used {most_used['Count']} times")

all_paints.sort_values(by=['Process Type', 'Count'], ascending=[True, False])

## 4 .Valores faltantes

### 4.1 Suma de nulos por variable 

#### 4.1.1 Análisis de consumo de pintura

En el resultado podemos observar cómo no existe algún valor nulo dentro de las variables que seleccionamos para esta base de datos, estos son datos que son necesarios dentro del proceso de pintado.

In [38]:
# Calcular el total de valores nulos para cada columna
paint_null_values= paint_df.isnull().sum()

print("paint_df , Null values per column:")                             
paint_null_values

#### 4.1.2 Pinturas y revestidos

En el resultado podemos observar cómo no existe algún valor nulo dentro de las variables que seleccionamos para esta base de datos, estos son datos que son necesarios en la evaluación del proceso de las pinturas. En donde se identifica datos importantes de la pintura utilizada como proveedor y precio.

In [39]:
# Calcular el total de valores nulos para cada columna
coating_null_values= coating_df.isnull().sum()

print("coating_df, Null values per column:")
coating_null_values                                                            

#### 4.1.3 Resumen de producción

En el resultado podemos observar cómo aquí si existen valores nulos en algunas variables de la base de datos. Esto puede llegar a pasar porque no se realizó el proceso necesario o hasta no se guardó el dato usado.

In [40]:
# Calcular el total de valores nulos para cada columna
production_null_values= production_df.isnull().sum()

print("production_df, Null values per column:")
production_null_values

#### 4.1.4 Defectos de producción

En este resultado podemos ver como la mayoría de las columnas tienen nulos, esto se debe a que las pinturas no fueron detectadas con algún defecto dentro del proceso de la revisión.

In [41]:
# Calcular el total de valores nulos para cada columna
defects_null_values= defects_df.isnull().sum()

print("defects_df, null values per column:")
defects_null_values

### 4.2 Indentificacion de columnas con mayor porcentaje de 15% datos nulos

El código calcula primero las filas del dataframe, después calcula el porcentaje de los valores nulos por cada columna y finalmente filtra los datos con el porcentaje seleccionado (en este caso 15%).  Si identifica valores mayores a 15% siendo nulos, te imprime las columnas, de no ser así imprime un mensaje comentando que no hay nulos arriba del %15.

#### 4.2.1 Análisis de consumo de pintura

In [42]:
total = paint_df.shape[0]

null_values_percentage= (paint_df.isnull().sum() / total) * 100

total_nulls= null_values_percentage[null_values_percentage > 15]
if not total_nulls.empty:
    print("Columns with more than 15% null data:" ,total_nulls)
else:
    print("No columns above 15 percent null data")

#### 4.2.2 Pinturas y revestidos

In [43]:
total = coating_df.shape[0]

null_values_percentage= (coating_df.isnull().sum() / total) * 100

total_nulls= null_values_percentage[null_values_percentage > 15]
if not total_nulls.empty:
    print("Columns with more than 15% null data:" ,total_nulls)
else:
    print("No columns above 15 percent null data")

#### 4.2.3 Resumen de producción

In [44]:
total = production_df.shape[0]

null_values_percentage= (production_df.isnull().sum() / total) * 100

total_nulls= null_values_percentage[null_values_percentage > 15]
if not total_nulls.empty:
    print("Columns with more than 15% null data: \n" ,total_nulls)
else:
    print("No columns above 15 percent null data ")

#### 4.2.4 Defectos de producción

In [45]:
total = defects_df.shape[0]

null_values_percentage= (defects_df.isnull().sum() / total) * 100

total_nulls= null_values_percentage[null_values_percentage > 15]
if not total_nulls.empty:
    print("Columns with more than 15% null data: \n" ,total_nulls)
else:
    print("No columns above 15 percent null data ")

## 5 Registros duplicados

Para cada una de los dataframes se utilizó la función de duplicated para identificar las filas duplicadas en cada uno de ellos.

#### 5.1 Resumen de producción

In [46]:
production_df[production_df.duplicated()]

#### 5.2 Análisis de consumo de pintura

In [47]:
paint_df[paint_df.duplicated()]

#### 5.3 Pinturas y revestidos

In [48]:
defects_df[defects_df.duplicated(keep=False)]                       

In [49]:
coating_df[coating_df.duplicated()]       

## Contribución individual

#### Hiram Muñoz
Construcción del _script_ de procesamiento de datos, verificación de tipos de datos, selección y eliminación de columnas innecesarias.
#### Raúl Murillo 
Verificación de la columna de diferencia de las columnas real y teórico y de la verificación de los metros cuadrados reales.

#### Andrea Garza 
Verificación del rendimiento real y diferencia con el rendimiento estándar, Calculo e identificación de diferencias en el promedio de Ancho y Espesor e Identificación de valores faltantes.

#### Erick Hernández 
Calculo de nuevo rendimiento std, identificación de renglones duplicados y verificación de metros cuadrados reales 

#### David Martínez
Cambio de nombres de variables y analizar información sobre pinturas repetidas en los renglones de producción (encontrados duplicados y segmentadas pinturas que aparecen por cada proceso)

#### Distribucion de Tareas: 
https://docs.google.com/spreadsheets/d/1Nx3N0PPtPqMVHnxO9pkpvJUUQX7RMOTvP8N3Cbafq70/edit?usp=sharing 

## Referencias
Apache Software Foundation. (s. f.). Feather File Format — Apache Arrow v15.0.1. Recuperado 12 de marzo de 2024, de https://arrow.apache.org/docs/python/feather.html